# Neural Networks: Part 2 - Model

In [ ]:
import torch
import pandas as pd
import numpy as np
import tqdm
import matplotlib.pyplot as plt

## Classifier Model

Restart your kernel, so nothing from above carries over.

First, go look at `model/model.py`. The model there should be familiar to you, it is the regression model from the prior lab. It has been updated to be flexibly defined according to the values we pass it.

In [ ]:
# this is a relative import, it looks in the directory 'model', then looks in the .py script 'networks' for our class
from model.networks import FullyConnectedNetwork

# instantiate a model
model = # write the command here to instantiate it

# confirm it has been made to your specifications
model

We can continue to customize our network as we go along. However, the current model produces an output designed for regression... how do we modify it to be used for classification?

- The model needs to produce a certain size of outputs. For a binary classification task, a single value, and for multi-class tasks, one value per class. 
- The model's output needs to be transformed to be in the range of 0 to 1.

Make your modifications now.

In [ ]:
# here is my model I made
from model.networks_Solution import FCNClassifier

model = FCNClassifier(17, 64, 1)
model

### Train

Let's put it all together and train a classifier on your processed dataset, using your very own dataloader

In [ ]:
from model.dataloader_Solution import CustomDataloader

# load data
data_dict = torch.load('data/wages_processed.pt')

# instantiate your dataloaders
train_dataloader = CustomDataloader(data_dict['x_train'],data_dict['y_train'], batch_size = 8, randomize=True)
val_dataloader = CustomDataloader(data_dict['x_val'],data_dict['y_val'], batch_size = 64)
test_dataloader = CustomDataloader(data_dict['x_test'],data_dict['y_test'], batch_size = 64)

I've put in an example way to set up your training loop, adapt it accordingly to your own dataloader and fill in the remaining code

In [ ]:
# instantiate your model with the correct inputs and outputs, and a reasonable hidden dim
model = FCNClassifier(17, 256, 1)

# instantiate your optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# log your losses
train_losses = []
val_losses = []

# define how many updates to your model
epochs = 100

# define your loss function for a binary classification class
# this is a binary cross entropy loss function, and it expects data in the range 0-1
loss_fn = torch.nn.BCELoss()

for _ in tqdm.tqdm(range(epochs)):
    losses = []
    for _ in range(train_dataloader.num_batches_per_epoch):
        # training data forward pass
        optimizer.zero_grad()
        train_batch = train_dataloader.fetch_batch()
        yhat = model(train_batch['x_batch'])
        train_loss = loss_fn(yhat, train_batch['y_batch'])

        # training data backward pass
        train_loss.backward()
        optimizer.step()
        losses.append(train_loss.detach().numpy())
    train_losses.append(np.mean(losses))

    
    losses = []
    for _ in range(val_dataloader.num_batches_per_epoch):
        # validation data forward pass only
        val_batch = val_dataloader.fetch_batch()
        yhat = model(val_batch['x_batch'])
        val_loss = loss_fn(yhat, val_batch['y_batch'])
        losses.append(val_loss.detach().numpy())
    val_losses.append(np.mean(losses))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(0, figsize = (12,6))
plt.title('Loss per epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(train_losses)
plt.plot(val_losses)
plt.legend(["Training loss","Validation loss"])

**Q: How to interpret these loss values? What would be the loss if we randomly chose 1s or 0s? Establish what your random baseline loss is, and see if you are beating it.**

Once you are satisfied with your classifier, don't forget to check how you do on your test data

In [ ]:
# test data performance
